In [66]:
pos_path = "/content/Positive_dataset.fasta"
neg_path = "/content/Negative_dataset.fasta"
prediction_file_path = "/content/candidate_dataset.fasta"

In [67]:
!pip install biopython
!pip install propy
!pip install propy3

In [68]:
import numpy as np
import pandas as pd
import matplotlib as plt
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_score, cross_val_predict
from sklearn.metrics import confusion_matrix
from sklearn.metrics import precision_recall_fscore_support


In [69]:
def read_fasta(file_path):
    sequences = {}
    current_sequence = None

    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()

            if line.startswith('>'):
                current_sequence = line[1:]
                sequences[current_sequence] = ''
            else:
                sequences[current_sequence] += line

    return sequences

In [70]:
def protparam_features(path):
  import csv
  from Bio import SeqIO
  from Bio.SeqUtils.ProtParam import ProteinAnalysis

  fasta_file = path
  csv_file = f"{path}_prot.csv"

  feature_list = []

  with open(fasta_file, "r") as handle:
      for record in SeqIO.parse(handle, "fasta"):
          accession_id = record.id
          sequence = str(record.seq)

          if 'X' not in sequence:
              protein_analysis = ProteinAnalysis(sequence)

              molecular_weight = protein_analysis.molecular_weight()
              pI = protein_analysis.isoelectric_point()
              amino_acid_composition = protein_analysis.count_amino_acids()
              aromaticity = protein_analysis.aromaticity()
              secondary_structure_fraction = protein_analysis.secondary_structure_fraction()
              gravy = protein_analysis.gravy()
              instability_index = protein_analysis.instability_index()
              flexibility = protein_analysis.flexibility()

              feature_list.append({
                  "Accession ID": accession_id,
                  "Sequence": sequence,
                  "Molecular Weight": molecular_weight,
                  "Isoelectric Point (pI)": pI,
                  "Amino Acid Composition": amino_acid_composition,
                  "Aromaticity": aromaticity,
                  "Secondary Structure Fraction": secondary_structure_fraction,
                  "GRAVY": gravy,
                  "Instability Index": instability_index,

              })

  with open(csv_file, mode="w", newline="") as csv_file:
      fieldnames = [
          "Accession ID", "Sequence", "Molecular Weight", "Isoelectric Point (pI)",
          "Amino Acid Composition", "Aromaticity", "Secondary Structure Fraction",
          "GRAVY", "Instability Index"
      ]
      writer = csv.DictWriter(csv_file, fieldnames=fieldnames)

      writer.writeheader()

      for feature in feature_list:
          writer.writerow(feature)
  print("CSV file containing protein features and Accession IDs has been created:", csv_file)

In [71]:
def propy3_features(path):
  from propy.CTD import CalculateC as calC, CalculateT as calT, CalculateD as calD

  data = "".join(open(f"{path}").readlines()).split(">")
  out = open(f"{path}_propy.csv", "w", encoding="UTF-8")
  feature = []

  c, t, d = '', '', ''
  for seq in data[1:]:
      dataFeature = []
      lines = seq.split("\n")
      header = lines[0]
      sequence = "".join(lines[1:])

      if 'X' in sequence:
          continue

      if '|' in header:
          identifier = header.split('|')[1]
      else:
          identifier = header

      dictC = calC(sequence)
      dictT = calT(sequence)
      dictD = calD(sequence)
      c, t, d = dictC.keys(), dictT.keys(), dictD.keys()
      dataFeature.append(identifier)
      for C in dictC:
          dataFeature.append(str(dictC[C]))
      for T in dictT:
          dataFeature.append(str(dictT[T]))
      for D in dictD:
          dataFeature.append(str(dictD[D]))
      feature.append(dataFeature)

  out.write("acc," + ",".join(list(c) + list(t) + list(d)) + "\n")
  for f in feature:
      out.write(",".join(f) + "\n")
  out.close()
  print("csv file generated")


In [72]:
protparam_features(neg_path)
protparam_features(pos_path)

CSV file containing protein features and Accession IDs has been created: <_io.TextIOWrapper name='/content/Negative_dataset.fasta_prot.csv' mode='w' encoding='UTF-8'>
CSV file containing protein features and Accession IDs has been created: <_io.TextIOWrapper name='/content/Positive_dataset.fasta_prot.csv' mode='w' encoding='UTF-8'>


In [73]:
propy3_features(pos_path)
propy3_features(neg_path)

csv file generated
csv file generated


In [74]:
def concat_data(path1, path2):

    data1_pos = pd.read_csv(path1)
    data2_pos = pd.read_csv(path2)

    merged_data = pd.concat([data1_pos, data2_pos], axis=1)

    merged_data = merged_data.drop(['acc', 'Accession ID', 'Sequence'], axis=1)

    return merged_data

In [75]:
positive_df = concat_data(f'{pos_path}_propy.csv',
                     f'{pos_path}_prot.csv')

In [76]:
positive_df['target'] = 1
positive_df.shape

(1785, 155)

In [77]:
negative_df = concat_data(f'{neg_path}_propy.csv'
            ,f'{neg_path}_prot.csv')

In [78]:
negative_df['target'] = 0
negative_df.shape

(8924, 155)

In [79]:
result = pd.concat([positive_df, negative_df], ignore_index=True)

In [80]:
result

,_PolarizabilityC1,_PolarizabilityC2,_PolarizabilityC3,_SolventAccessibilityC1,_SolventAccessibilityC2,_SolventAccessibilityC3,_SecondaryStrC1,_SecondaryStrC2,_SecondaryStrC3,_ChargeC1,...,_HydrophobicityD3075,_HydrophobicityD3100,Molecular Weight,Isoelectric Point (pI),Amino Acid Composition,Aromaticity,Secondary Structure Fraction,GRAVY,Instability Index,target
0,0.293,0.413,0.294,0.421,0.283,0.296,0.379,0.337,0.285,0.115,...,75.717,100.000,59908.9278,8.092346,"{'A': 38, 'C': 14, 'D': 32, 'E': 25, 'F': 25, ...",0.124283,"(0.265774378585086, 0.28489483747609945, 0.376...",-0.313193,53.047648,1
1,0.290,0.462,0.247,0.427,0.328,0.245,0.484,0.239,0.277,0.102,...,73.656,99.462,41647.3781,5.138722,"{'A': 31, 'C': 3, 'D': 19, 'E': 32, 'F': 18, '...",0.086022,"(0.3817204301075269, 0.2768817204301075, 0.336...",-0.253763,32.184946,1
2,0.062,0.312,0.625,0.312,0.375,0.312,0.562,0.312,0.125,0.250,...,56.250,93.750,2174.5917,10.266413,"{'A': 0, 'C': 0, 'D': 0, 'E': 1, 'F': 1, 'G': ...",0.250000,"(0.3125, 0.125, 0.375)",-0.937500,77.300625,1
3,0.299,0.444,0.257,0.414,0.314,0.272,0.456,0.284,0.261,0.111,...,74.330,100.000,29365.3882,8.893817,"{'A': 16, 'C': 5, 'D': 6, 'E': 18, 'F': 15, 'G...",0.091954,"(0.32567049808429116, 0.26053639846743293, 0.3...",-0.244828,43.105747,1
4,0.252,0.470,0.278,0.348,0.296,0.357,0.339,0.470,0.191,0.113,...,73.913,99.130,13569.4919,8.807364,"{'A': 1, 'C': 3, 'D': 3, 'E': 7, 'F': 8, 'G': ...",0.130435,"(0.2173913043478261, 0.19130434782608693, 0.51...",-0.323478,39.008696,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10704,0.378,0.378,0.244,0.356,0.311,0.333,0.600,0.200,0.200,0.111,...,48.889,100.000,5001.5248,4.715272,"{'A': 7, 'C': 1, 'D': 1, 'E': 8, 'F': 0, 'G': ...",0.044444,"(0.4666666666666667, 0.2, 0.2444444444444444)",-0.540000,60.180000,0
10705,0.250,0.406,0.344,0.359,0.375,0.266,0.656,0.156,0.188,0.141,...,67.188,98.438,7545.6283,6.394239,"{'A': 8, 'C': 0, 'D': 4, 'E': 6, 'F': 1, 'G': ...",0.046875,"(0.453125, 0.1875, 0.265625)",-0.510938,45.356250,0
10706,0.480,0.280,0.240,0.440,0.320,0.240,0.360,0.240,0.400,0.080,...,80.000,100.000,2702.8911,7.953481,"{'A': 4, 'C': 1, 'D': 1, 'E': 0, 'F': 0, 'G': ...",0.120000,"(0.24000000000000002, 0.39999999999999997, 0.2)",-0.840000,36.872000,0
10707,0.389,0.361,0.250,0.491,0.361,0.148,0.509,0.194,0.296,0.130,...,74.074,100.000,11993.1732,4.836566,"{'A': 16, 'C': 0, 'D': 10, 'E': 9, 'F': 6, 'G'...",0.101852,"(0.35185185185185186, 0.2962962962962963, 0.28...",-0.518519,49.854630,0


In [81]:
result.shape

(10709, 155)

In [82]:
def data_processing(df):
    df['Amino Acid Composition'] = df['Amino Acid Composition'].apply(eval)
    expanded_ac = df['Amino Acid Composition'].apply(pd.Series)
    df = pd.concat([df.drop(['Amino Acid Composition'], axis=1), expanded_ac], axis=1)
    df['Secondary Structure Fraction'] = df['Secondary Structure Fraction'].apply(eval)
    df[['Fraction_1', 'Fraction_2', 'Fraction_3']] = pd.DataFrame(df['Secondary Structure Fraction'].tolist(), index=df.index)
    df = df.drop(['Secondary Structure Fraction'], axis=1)
    return df


In [83]:
result = data_processing(result)

In [84]:
result

,_PolarizabilityC1,_PolarizabilityC2,_PolarizabilityC3,_SolventAccessibilityC1,_SolventAccessibilityC2,_SolventAccessibilityC3,_SecondaryStrC1,_SecondaryStrC2,_SecondaryStrC3,_ChargeC1,...,Q,R,S,T,V,W,Y,Fraction_1,Fraction_2,Fraction_3
0,0.293,0.413,0.294,0.421,0.283,0.296,0.379,0.337,0.285,0.115,...,11,29,31,28,41,15,25,0.265774,0.284895,0.376673
1,0.290,0.462,0.247,0.427,0.328,0.245,0.484,0.239,0.277,0.102,...,18,13,21,13,20,3,11,0.381720,0.276882,0.336022
2,0.062,0.312,0.625,0.312,0.375,0.312,0.562,0.312,0.125,0.250,...,1,3,0,0,0,1,2,0.312500,0.125000,0.375000
3,0.299,0.444,0.257,0.414,0.314,0.272,0.456,0.284,0.261,0.111,...,16,12,24,16,14,1,8,0.325670,0.260536,0.363985
4,0.252,0.470,0.278,0.348,0.296,0.357,0.339,0.470,0.191,0.113,...,6,5,4,19,7,1,6,0.217391,0.191304,0.513043
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10704,0.378,0.378,0.244,0.356,0.311,0.333,0.600,0.200,0.200,0.111,...,0,5,4,3,3,0,2,0.466667,0.200000,0.244444
10705,0.250,0.406,0.344,0.359,0.375,0.266,0.656,0.156,0.188,0.141,...,2,7,3,1,6,1,1,0.453125,0.187500,0.265625
10706,0.480,0.280,0.240,0.440,0.320,0.240,0.360,0.240,0.400,0.080,...,2,1,3,1,1,2,1,0.240000,0.400000,0.200000
10707,0.389,0.361,0.250,0.491,0.361,0.148,0.509,0.194,0.296,0.130,...,4,12,3,2,5,2,3,0.351852,0.296296,0.287037


In [85]:
result.shape

(10709, 176)

In [86]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import  accuracy_score

In [87]:
y = result['target']
X = result.drop(['target'], axis=1)

In [88]:
from sklearn.metrics import matthews_corrcoef
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,stratify= y ,random_state=42)

model3 = RandomForestClassifier(random_state=42)
model3.fit(X_train, y_train)

y_pred3 = model3.predict(X_test)

precision3 = precision_score(y_test, y_pred3)
recall3 = recall_score(y_test, y_pred3)
f1_3 = f1_score(y_test, y_pred3)
score_sc3 = round(accuracy_score(y_pred3,y_test)*100,2)

cv_scores3 = cross_val_score(model3, X_train, y_train, cv=5)
mean_cv_score3 = np.mean(cv_scores3)

y_pred_cv3 = cross_val_predict(model3, X_train, y_train, cv=5)


conf_matrix3 = confusion_matrix(y_train, y_pred_cv3)
precision_cv3, recall_cv3, f1_score_cv3, _ = precision_recall_fscore_support(y_train, y_pred_cv3, average='weighted')
tn_cv3, fp_cv3, fn_cv3, tp_cv3 = conf_matrix3.ravel()
sensitivity_cv3 = tp_cv3 / (tp_cv3 + fn_cv3)
specificity_cv3 = tn_cv3 / (tn_cv3 + fp_cv3)

y_prob = model3.predict_proba(X_test)
y_prob_positive = y_prob[:, 1]
y_prob_positive
roc_3 = roc_auc_score(y_test, y_prob_positive)

mcc_score3 = matthews_corrcoef(y_test, y_pred3)
auc_score3 = roc_auc_score(y_test, y_prob_positive)


In [90]:
#comment out if want to go through the feature file of propy and protparam
file_to_delete = f'{neg_path}_propy.csv'
!rm {file_to_delete}
file_to_delete = f'{neg_path}_prot.csv'
!rm {file_to_delete}
file_to_delete = f'{pos_path}_propy.csv'
!rm {file_to_delete}
file_to_delete = f'{pos_path}_prot.csv'
!rm {file_to_delete}

In [103]:
print("f1 Score" ,f1_3)
print("accuracy",score_sc3)
print("false negative",fn_cv3)
print("false positive",fp_cv3)
print("True positive",tp_cv3)
print("True negative",tn_cv3)
print("roc",roc_3)
print("sensitivity: ",sensitivity_cv3*100)
print("Specificity: ",specificity_cv3*100)
print("Matthews Correlation Coefficient (MCC):", mcc_score3)
print("Area Under the Curve (AUC): ",(auc_score3))
print("Total data points" , 10709)
print("Negative data points", 8924)
print("Positive data points", 1785)
print(conf_matrix3)

f1 Score 0.98
accuracy 99.35
false negative 68
false positive 1
True positive 1360
True negative 7138
roc 0.9983248201241282
sensitivity:  95.23809523809523
Specificity:  99.98599243591539
Matthews Correlation Coefficient (MCC): 0.9763746171404237
Area Under the Curve (AUC):  0.9983248201241282
Total data points 10709
Negative data points 8924
Positive data points 1785
[[7138    1]
 [  68 1360]]


In [92]:
def pred_An(inp):
  fasta_sequences = read_fasta(inp)
  headers = []
  for header in fasta_sequences.items():
    headers.append(header)

  propy3_features(inp)
  protparam_features(inp)
  prediction_data_1 = pd.read_csv(f'{inp}_propy.csv')
  prediction_data_2 = pd.read_csv(f'{inp}_prot.csv')
  merged_data = concat_data(f'{inp}_propy.csv',f'{inp}_prot.csv')

  final_data = data_processing(merged_data)

  print(final_data.shape)

  prediction = model3.predict(final_data)
  result_df = pd.DataFrame({'Headers' : headers,'Prediction': prediction})
  file_to_delete = f'{inp}_propy.csv'
  !rm {file_to_delete}
  file_to_delete = f'{inp}_prot.csv'
  !rm {file_to_delete}
  return result_df

In [93]:
fasta_file_path = prediction_file_path
fasta_sequences = read_fasta(fasta_file_path)

for header, sequence in fasta_sequences.items():
    print(f'Header: {header}')
    print(f'Sequence: {sequence}\n')

Header: sp|A6QIG7|CHIPS_STAAE Chemotaxis inhibitory protein OS=Staphylococcus aureus (strain Newman) OX=426430 GN=chp PE=1 SV=1
Sequence: MKKKLATTVLALSFLTAGISTHHHSAKAFTFEPFPTNEEIESNKKMLEKEKAYKESFKNSGLPTTLGKLDERLRNYLKKGTKNSAQFEKMVILTENKGYYTVYLNTPLAEDRKNVELLGKMYKTYFFKKGESKSSYVINGPGKTNEYAY

Header: sp|O60602|TLR5_HUMAN Toll-like receptor 5 OS=Homo sapiens OX=9606 GN=TLR5 PE=1 SV=4
Sequence: MGDHLDLLLGVVLMAGPVFGIPSCSFDGRIAFYRFCNLTQVPQVLNTTERLLLSFNYIRTVTASSFPFLEQLQLLELGSQYTPLTIDKEAFRNLPNLRILDLGSSKIYFLHPDAFQGLFHLFELRLYFCGLSDAVLKDGYFRNLKALTRLDLSKNQIRSLYLHPSFGKLNSLKSIDFSSNQIFLVCEHELEPLQGKTLSFFSLAANSLYSRVSVDWGKCMNPFRNMVLEILDVSGNGWTVDITGNFSNAISKSQAFSLILAHHIMGAGFGFHNIKDPDQNTFAGLARSSVRHLDLSHGFVFSLNSRVFETLKDLKVLNLAYNKINKIADEAFYGLDNLQVLNLSYNLLGELYSSNFYGLPKVAYIDLQKNHIAIIQDQTFKFLEKLQTLDLRDNALTTIHFIPSIPDIFLSGNKLVTLPKINLTANLIHLSENRLENLDILYFLLRVPHLQILILNQNRFSSCSGDQTPSENPSLEQLFLGENMLQLAWETELCWDVFEGLSHLQVLYLNHNYLNSLPPGVFSHLTALRGLSLNSNRLTVLSHNDLPANLEILDISRNQLLAPNPDVFVSLSVLDITHNKFICECELSTFINWLNHTNVTIAGPPADIY

In [94]:
answers = pred_An(prediction_file_path)

csv file generated
CSV file containing protein features and Accession IDs has been created: <_io.TextIOWrapper name='/content/candidate_dataset.fasta_prot.csv' mode='w' encoding='UTF-8'>
(37, 175)


In [95]:
print(len(answers))

37


In [96]:
print (answers)

                                              Headers  Prediction
0   (sp|A6QIG7|CHIPS_STAAE Chemotaxis inhibitory p...           1
1   (sp|O60602|TLR5_HUMAN Toll-like receptor 5 OS=...           1
2   (sp|P09429|HMGB1_HUMAN High mobility group pro...           1
3   (sp|P0C8E7|EVA1_RHISA Evasin-1 OS=Rhipicephalu...           0
4   (sp|P35408|PE2R4_HUMAN Prostaglandin E2 recept...           1
5   (sp|P84189|NDBH1_TITSE Hypotensin-1 OS=Tityus ...           1
6   (sp|Q6P589|TP8L2_HUMAN Tumor necrosis factor a...           1
7   (sp|Q8NFZ5|TNIP2_HUMAN TNFAIP3-interacting pro...           1
8   (sp|Q99788|CML1_HUMAN Chemerin-like receptor 1...           1
9   (sp|Q99MT6|SUCR1_MOUSE Succinate receptor 1 OS...           1
10  (sp|Q9BSK4|FEM1A_HUMAN Protein fem-1 homolog A...           1
11  (sp|Q9BXA5|SUCR1_HUMAN Succinate receptor 1 OS...           1
12  (sp|Q9EQ14|IL23A_MOUSE Interleukin-23 subunit ...           1
13  (sp|P0C8E8|EVA3_RHISA Evasin-3 OS=Rhipicephalu...           1
14  (sp|Q6

In [97]:
answers.to_csv(f'{prediction_file_path}_ans.csv')

In [98]:
answers['Prediction'].value_counts()

Prediction
1    30
0     7
Name: count, dtype: int64

In [99]:
answers_with_value_0 = answers[answers['Prediction'] == 0]

In [100]:
answers_with_value_0

,Headers,Prediction
3,(sp|P0C8E7|EVA1_RHISA Evasin-1 OS=Rhipicephalu...,0
20,(tr|A0A6S4PZK0|A0A6S4PZK0_9ASTR Elongation fac...,0
21,(tr|A0A6S4Q3D4|A0A6S4Q3D4_9ASTR Elongation fac...,0
22,(tr|A0A6S4Q3R2|A0A6S4Q3R2_9ASTR Elongation fac...,0
23,(tr|A0A6S4Q8Q2|A0A6S4Q8Q2_ATRLA Elongation fac...,0
24,(tr|A0A6S4Q948|A0A6S4Q948_9ASTR Elongation fac...,0
25,(tr|A0A6S4QC37|A0A6S4QC37_ATRLA Elongation fac...,0
